In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import LambdaLR
from sklearn.metrics import accuracy_score


from sklearn.ensemble import RandomForestClassifier


# Import evening / morning file

In [ ]:
path_to_folder = 'D:/DATA_THESIS/Projekt_nw_AG_AJ/Outputs_project_data_preprocessing'
path_to_folder_questionnaires = ''

In [ ]:
evening_morning_l = pd.read_pickle(f'{path_to_folder}/3_questionnaires_fitbit/m_e_sleeplength.pkl")

In [ ]:
evening_morning_l.columns

In [ ]:
evening_morning_l = evening_morning_l.drop([ 'logId',
                                            # 'dateOfSleep', 
                                            'startTime', 'endTime',
                                            'type', 'infoCode',
                                            'minutesAwake', 'timeInBed',
                                            'logType', 'levels', 'mainSleep','filledTimestamp_morning', 
                                            'filledTimestamp_evening', 'morning_date', 'time_difference',
                                             # 'sleepQuality', 'composure','rest',
                                              'id', 'minutesToFallAsleep', 'minutesAfterWakeup', 'duration', 'efficiency' ], axis=1)

In [ ]:
evening_morning_l

# Load questionaries pre

In [ ]:
pre_q = pd.read_csv('demography.csv')

In [ ]:
pre_q

In [ ]:
pre_q['patient_id'] = pre_q['UID']

In [ ]:
pre_q.columns

### usunięte
- skąd wiesz o badaniu - informacja dla nas
- ID - nic nie wnoszą liczby po kolei
- 'Jaka jest Twoja narodowość?' , 'Czy język polski jest Twoim pierwszym lub ojczystym językiem?' - informacja dla zespołu badawczego
- 'wiek' - już jest takie pytanie, poza tym zawiera NaN
- 'rozmiar polar' - dla zespołu
- 'Jak określasz swoją płeć?'- już jest pytanie
- 'telefon' - dla zespołu

In [ ]:
eseitals = pre_q[[ 
       'Jaki jest Twój wiek (w latach, liczbą)?',
       'Czy język polski jest Twoim pierwszym lub ojczystym językiem?',
       'Czy masz dzieci w wieku poniżej 18 lat?',
       'Jeśli masz dzieci, podaj proszę ich liczbę.',
       'Czy na co dzień sprawujesz opiekę nad dziećmi?',
       'Twoja sytuacja ekonomiczna w porównaniu do przeciętnej osoby w Twoim kraju jest:',
       'Jaki jest Twój status zatrudnienia?',
       'W jakim sektorze jesteś zatrudniony/a?',
       'Jaki jest Twój status związku?',
       'Ile osób łącznie z Tobą mieszka w gospodarstwie domowym?',
       'Jakie masz wykształcenie?', 
        'Czy obecnie studiujesz?',
       'Czy jesteś osobą religijną?',
       'Z jakimi poglądami politycznymi się utożsamiasz na osi ekonomicznej?',
       'Z jakimi poglądami politycznymi się utożsamiasz na osi światopoglądowej?',
        'waga', 'wzrost',  'płeć',
       'patient_id'
                 ]]

In [ ]:
import pandas as pd

# Assuming df1 and df2 are your two dataframes
# Merge the two dataframes on the "patient" column
merged_df = pd.merge(evening_morning_l, eseitals, on='patient_id', how='inner')


In [ ]:
merged_df

In [ ]:
merged_df['Twoja sytuacja ekonomiczna w porównaniu do przeciętnej osoby w Twoim kraju jest:'].unique()

## Manage non-numeric data

In [ ]:
# Get unique words from the column
def string_to_numeric(df= merged_df, column_name = 'Twoja sytuacja ekonomiczna w porównaniu do przeciętnej osoby w Twoim kraju jest:'):
    unique_words = df[column_name].unique()
    
    # Create a mapping dictionary
    mapping = {word: idx for idx, word in enumerate(unique_words)}
    
    # Replace words with numbers
    df[f'num_{column_name}'] = df[column_name].map(mapping)
    
    df = df.drop([column_name], axis = 1)
    
    return df

In [ ]:
non_numeric_columns = merged_df.select_dtypes(exclude=['number']).columns.tolist()
non_numeric_columns

In [ ]:
for num_col in [ 'Czy język polski jest Twoim pierwszym lub ojczystym językiem?',
 'Czy masz dzieci w wieku poniżej 18 lat?',
 'Jeśli masz dzieci, podaj proszę ich liczbę.',
 'Czy na co dzień sprawujesz opiekę nad dziećmi?',
 'Twoja sytuacja ekonomiczna w porównaniu do przeciętnej osoby w Twoim kraju jest:',
 'Jaki jest Twój status zatrudnienia?',
 'W jakim sektorze jesteś zatrudniony/a?',
 'Jaki jest Twój status związku?',
 'Ile osób łącznie z Tobą mieszka w gospodarstwie domowym?',
 'Jakie masz wykształcenie?',
 'Czy obecnie studiujesz?',
 'Czy jesteś osobą religijną?',
 'płeć']:
    merged_df = string_to_numeric(merged_df, num_col)

In [ ]:
merged_df.select_dtypes(exclude=['number']).columns.tolist()

In [ ]:
merged_df

In [ ]:
merged_df.columns[merged_df.isna().any()].tolist()
rows_with_nan = merged_df[merged_df.isna().any(axis=1)]

print("Rows with NaN values:")
rows_with_nan

In [ ]:
columns_with_nan = merged_df.columns[merged_df.isna().any()].tolist()
print("Columns with NaN values:")
print(columns_with_nan)

In [ ]:
nan_counts = merged_df.isna().sum()

# Total count of NaN values
total_nan_count = nan_counts.sum()

print("Total NaN values in the DataFrame:", total_nan_count)

In [ ]:
merged_df.to_csv(f'{path_to_folder}/6_emotions_aggregates/pre_questionaries_evening_morning.csv', index = False)

# Add more data => emotion questionaries

In [ ]:
import os
import json
from tqdm import tqdm
list_patient = []
list_id_patient = os.listdir(f"{path_to_folder_questionnaires}/ankiety")
list_of_data_for_all_patients = []

for iter in tqdm(list_id_patient):
#     print('Patient: ', patient )
    lista_id_iter = os.listdir(f'{path_to_folder_questionnaires}/ankiety/{iter}')
    for patient in tqdm(lista_id_iter):
    #     print('Patient: ', patient )
        lista_id_emocion_patient = os.listdir(f'{path_to_folder_questionnaires}/ankiety/{iter}/{patient}/assessments/emotion')
        for emotion in lista_id_emocion_patient:
    #         print("Emotion: ", emotion)
            f = open(f'{path_to_folder_questionnaires}/ankiety/{iter}/{patient}/assessments/emotion/{emotion}', encoding="utf8")
            data = json.load(f)
            list_of_data_for_all_patients.append(data)
            list_patient.append(patient)

In [ ]:
list_id_patient

In [ ]:
import pandas as pd
from glob import glob

df = pd.DataFrame()
for iter in tqdm(list_id_patient):
    lista_id_iter = os.listdir(f'C:/Users/Admin/Thesis/data/ankiety/{iter}')
    for patient in tqdm(lista_id_iter):
        patient_data = pd.concat([pd.read_json(f_name, lines=True) for f_name in glob(f'C:/Users/Admin/Thesis/data/ankiety/{iter}/{patient}/assessments/emotion/*.json')])
        patient_data['patient_id'] = patient
        df = pd.concat([df, patient_data])

In [ ]:
emotions_df = df.copy().reset_index(drop=True)
emotions_df

In [ ]:
emotions_df['emotionTimestamp'][25192]


In [ ]:
emotions_df['emotionTimestamp'] = emotions_df['emotionTimestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f') if '.' in x else datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))

emotions_df['date'] = pd.to_datetime(emotions_df['emotionTimestamp'])
emotions_df['hour'] = emotions_df['date'].dt.hour
emotions_df['dateOfSleep'] = pd.to_datetime(emotions_df['date']).dt.floor('D')

emotions_df['dateOfSleep'] = emotions_df['dateOfSleep'] + pd.Timedelta(days=1)

In [ ]:
emotions_df

In [ ]:
result_df = emotions_df.copy() 

contains_dict = result_df.map(lambda x: isinstance(x, dict)).any()
print("Columns containing dictionaries:")
print(contains_dict)

columns_with_dicts = contains_dict[contains_dict].index.tolist()
print("Columns containing dictionaries:")
print(columns_with_dicts)


list_of_columns_with_dict = columns_with_dicts
for column in list_of_columns_with_dict:
    # Expanding predictions
#     display(result_df.head(3))
    result_df[column].fillna({}, inplace=True)
    expanded_df = pd.json_normalize(result_df[column])
    result_df.reset_index(drop=True, inplace=True)
    expanded_df.reset_index(drop=True, inplace=True)
    result_df = pd.concat([result_df, expanded_df], axis=1)
    result_df = result_df.drop(column, axis=1)

In [ ]:
result_df.columns

In [ ]:
emotions_df = result_df.copy()

In [ ]:
emotions_df

In [ ]:
emotions_df['intenseEmotion'].unique()

## join features in one column

In [ ]:
unique_values = {np.nan: 0, 'DONT_KNOW': 1, 'NO': 2, 'YES': 3}
emotions_df.replace({'intenseEmotion': unique_values}, inplace=True)

In [ ]:
emotions_df['intenseEmotion'].unique()

In [ ]:
# replace NAN with max_value+1
max_values = emotions_df[['intenseEmotion', 'valence', 'arousal']].max(axis=0) + 1
emotions_df.fillna(max_values, inplace=True)

In [ ]:
emotions_df

In [ ]:
emotions_df['features'] = emotions_df[['intenseEmotion', 'valence', 'arousal']].values.tolist()

In [ ]:
emotions_df

In [ ]:
grouped_df = emotions_df.groupby(['patient_id', 'dateOfSleep'])[['intenseEmotion', 'valence', 'arousal', 'hour']].apply(lambda x: x.values.tolist()).reset_index()

In [ ]:
grouped_df[0][0][0][0]

In [ ]:
len(list(grouped_df['patient_id'].unique()))

In [ ]:
grouped_df.rename(columns={0: 'features'}, inplace=True)

In [ ]:
grouped_df

In [ ]:
# grouped_df.to_pickle('features_emotion_hour.plk')

In [ ]:
len(list(emotions_df['patient_id'].unique()))

In [ ]:
emotions_df = emotions_df.drop(['features'], axis=1)

## Merge with date

In [ ]:
evening_morning_l = pd.read_pickle("3_questionnaires_fitbit/m_e_sleeplength.pkl")

In [ ]:
evening_morning_l = evening_morning_l.drop([ 'logId', 'startTime', 'endTime',
                                            'type', 'infoCode',
                                            'minutesAwake', 'timeInBed',
                                            'logType', 'levels', 'mainSleep','filledTimestamp_morning', 
                                            'filledTimestamp_evening', 'morning_date', 'time_difference',
                                             'sleepQuality', 'composure','rest' , 'id', 'minutesToFallAsleep', 'minutesAfterWakeup', 'duration', 'efficiency' ], axis=1)

In [ ]:
merged_df = pd.merge(evening_morning_l, eseitals, on='patient_id', how='inner')

In [ ]:
merged_df

In [ ]:
merged_df['dateOfSleep'] = pd.to_datetime(merged_df['dateOfSleep'])
emotions_df['dateOfSleep'] = pd.to_datetime(emotions_df['dateOfSleep'])
# emotions_df = emotions_df[['features', 'dateOfSleep', 'patient_id']]
merged_with_emotion = pd.merge(emotions_df, merged_df, on=['patient_id','dateOfSleep' ], how='inner')
# merged_with_emotion = pd.merge(merged_with_emotion, grouped_df, on=['patient_id','dateOfSleep' ], how='inner')

In [ ]:
emotions_df.columns

In [ ]:
merged_with_emotion[['emotionTimestamp', 'dateOfSleep']]

In [ ]:
non_numeric_columns = merged_with_emotion.select_dtypes(exclude=['number']).columns.tolist()
non_numeric_columns

In [ ]:
non_num_col = [
    # 'Jak określasz swoją płeć?',
 'Twoja sytuacja ekonomiczna w porównaniu do przeciętnej osoby w Twoim kraju jest:',
 'Ile osób łącznie z Tobą mieszka w gospodarstwie domowym?',
 'Czy obecnie studiujesz?']
for num_col in non_num_col:
    merged_with_emotion = string_to_numeric(merged_with_emotion, num_col)

In [ ]:
merged_with_emotion.select_dtypes(exclude=['number']).columns.tolist()

In [ ]:
merged_with_emotion

In [ ]:
merged_with_emotion.columns[merged_with_emotion.isna().any()].tolist()
rows_with_nan = merged_with_emotion[merged_with_emotion.isna().any(axis=1)]

print("Rows with NaN values:")
rows_with_nan

In [ ]:
columns_with_nan = merged_with_emotion.columns[merged_with_emotion.isna().any()].tolist()
print("Columns with NaN values:")
print(columns_with_nan)

In [ ]:
merged_with_emotion

In [ ]:
merged_with_emotion

In [ ]:
merged_with_emotion.to_csv(f'{path_to_folder}/6_emotions_aggregates/emotion_evening_morning.csv', index=False)